## 1. Prueba no paramétrica con Kruskal-Wallis
La prueba de Kruskal-Wallis es una prueba no paramétrica y una alternativa al ANOVA. Por no paramétrica, se entiende que no se supone que los datos procedan de una distribución determinada. El objetivo principal de esta prueba es determinar si existe una diferencia estadística entre las medianas de dos grupos independientes. Sin embargo, empezaremos por una prueba comparando los 4 grupos para cada una de las variables independientes.

Hipótesis:
La prueba de Kruskal-Wallis tiene las hipótesis nula y alternativa que se exponen a continuación:

- La hipótesis nula (H0): La mediana es la misma para todos los grupos de datos.
- La hipótesis alternativa: (H1): La mediana no es igual para todos los grupos de datos.

https://www.geeksforgeeks.org/how-to-perform-a-kruskal-wallis-test-in-python/

Consideremos un ejemplo en el que queremos determinar si la frecuencia de apariencia de diferentes conceptos de anonimización provoca una diferencia en la clasificación del tipo de documento en un litigio de divorcio. Con base en la sección anterior, decidimos optar por grupos de 3 o 4 documentos de litigios en materia de divorcio de primera instancia de la Ciudad de México. Ahora, cada grupo se analizará a la luz de un concepto de anonimización. Se han calculado las frecuencia de apariencia de estos conceptos en el paso anterior.

### a: Crear los datos

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel('1_Descr_normal.xlsx', sheet_name='Transformados continuas', index_col=0)

elec_metodo = pd.read_excel('1_Descr_normal.xlsx', sheet_name='Elección método analisis').drop(['Distribución normal del par tipo / variable continua'], axis=1)
elec_metodo = elec_metodo.loc[elec_metodo['Prueba'] == 'KRUSKAL-WALLIS']
elec_metodo = elec_metodo.drop(['Prueba'], axis=1).reset_index(drop=True)

tipos = list(pd.unique(df['Y_type']))
variables = list(pd.unique(elec_metodo['Variable']))

groups = {}
for v in variables:
    groups[v] = {}

for t in tipos:
    doc = df.loc[df['Y_type'] == t]
    for v in variables:
        groups[v][t] = list(doc[v])

In [62]:
from scipy import stats

print('Necesitamos un p-value inferior a 0.05 para rechazar la hipótesis nula.\n')

# Empezamos por la comparación entre los 4 grupos
result = {}
tabla_verdad = {}
i = 0
for g in groups:
    g0 = groups[g][tipos[0]]
    g1 = groups[g][tipos[1]]
    g2 = groups[g][tipos[2]]
    g3 = groups[g][tipos[3]]
    result[g] = list(stats.kruskal(g0, g1, g2, g3))

    print(f'Para la variable {g}, obtenemos un estadístico de Kruskal-Wallis de {result[g][0]} con un p-value de {result[g][1]} entre los 4 tipos de documentos.')
    if result[g][1] < 0.05:
        print(f'Puesto que el p-value < 0.05, podemos rechazar la hipótesis nula para la varibale {g}. La variable {v} conduce a diferencias estadísticamente significativas entre los 4 tipos de documento de divorcio.\n')
        result[g].append(result[g][1]) #True
    elif result[g][1] >= 0.05:
        print(f'Puesto que el p-value >= 0.05, NO podemos rechazar la hipótesis nula para la varibale {g}. La variable {v} NO conduce a diferencias estadísticamente significativas entre los 4 tipos de documento de divorcio.\n')
        result[g].append(result[g][1]) #False
    i += 1

    for r in result:
        tabla_verdad[r] = result[r][2]

df_tabla_verdad = pd.DataFrame(tabla_verdad, index=[0,1,2,3,4,5,6,7,8,9])
df_tabla_verdad.to_excel('2_tabla_verdad.xlsx')

Necesitamos un p-value inferior a 0.05 para rechazar la hipótesis nula.

Para la variable X_apellido_actor, obtenemos un estadístico de Kruskal-Wallis de 9.008665713144007 con un p-value de 0.029175892749961824 entre los 4 tipos de documentos.
Puesto que el p-value < 0.05, podemos rechazar la hipótesis nula para la varibale X_apellido_actor. La variable X_num_juzgado conduce a diferencias estadísticamente significativas entre los 4 tipos de documento de divorcio.

Para la variable X_apellido_autorizado, obtenemos un estadístico de Kruskal-Wallis de 24.341693988752766 con un p-value de 2.119516782772439e-05 entre los 4 tipos de documentos.
Puesto que el p-value < 0.05, podemos rechazar la hipótesis nula para la varibale X_apellido_autorizado. La variable X_num_juzgado conduce a diferencias estadísticamente significativas entre los 4 tipos de documento de divorcio.

Para la variable X_apellido_demandado, obtenemos un estadístico de Kruskal-Wallis de 6.768958620847607 con un p-value de 0.

In [80]:
# Realizamos ahora la prueba entre tipos documentos, comparando cada vez un documento con el otro (60 comparaciones en total), 
# para examinar si hay diferencias estadísticamente significativas cuando comparamos sólamente dos documentos entre sí.
# El objetivo es descartar las variables que no permitiran diferenciar entre los documentos

result_binome = {}
for g in groups:
    result_binome[g] = {}

i = 0
for g in groups:
    g0 = groups[g][tipos[0]]
    g1 = groups[g][tipos[1]]
    g2 = groups[g][tipos[2]]
    g3 = groups[g][tipos[3]]
    pares = [[g0, g1, [tipos[0]], [tipos[1]]],
            [g0, g2, [tipos[0]], [tipos[2]]],
            [g0, g3, [tipos[0]], [tipos[3]]],
            [g1, g2, [tipos[1]], [tipos[2]]],
            [g1, g3, [tipos[1]], [tipos[3]]],
            [g2, g3, [tipos[2]], [tipos[3]]]]
    for p in pares:
        result_binome[g][f'{p[2]} y {p[3]}'] = list(stats.kruskal(p[0], p[1]))

        print(f'''{i}. Para la variable {g}, obtenemos un estadístico de Kruskal-Wallis de {result_binome[g][f'{p[2]} y {p[3]}'][0]} con un p-value de {result_binome[g][f'{p[2]} y {p[3]}'][1]} entre los tipos de documentos {p[2]} y {p[3]}.''')
        if result_binome[g][f'{p[2]} y {p[3]}'][1] < 0.05:
            print(f'Puesto que el p-value < 0.05, podemos rechazar la hipótesis nula para la varibale {g}. La variable {v} conduce a diferencias estadísticamente significativas entre los tipos de documentos {p[2]} y {p[3]}.\n')
            result_binome[g][f'{p[2]} y {p[3]}'].append(result_binome[g][f'{p[2]} y {p[3]}'][1]) #True
        elif result_binome[g][f'{p[2]} y {p[3]}'][1] >= 0.05:
            print(f'Puesto que el p-value >= 0.05, NO podemos rechazar la hipótesis nula para la varibale {g}. La variable {v} NO conduce a diferencias estadísticamente significativas entre los tipos de documentos {p[2]} y {p[3]}.\n')
            result_binome[g][f'{p[2]} y {p[3]}'].append(result_binome[g][f'{p[2]} y {p[3]}'][1]) #False
        i += 1

df_result = pd.DataFrame(result_binome)

df_result


0. Para la variable X_apellido_actor, obtenemos un estadístico de Kruskal-Wallis de 2.015810276679856 con un p-value de 0.15566814689657382 entre los tipos de documentos ['Solicitud de copias certificadas'] y ['Contestación de demanda'].
Puesto que el p-value >= 0.05, NO podemos rechazar la hipótesis nula para la varibale X_apellido_actor. La variable X_num_juzgado NO conduce a diferencias estadísticamente significativas entre los tipos de documentos ['Solicitud de copias certificadas'] y ['Contestación de demanda'].

1. Para la variable X_apellido_actor, obtenemos un estadístico de Kruskal-Wallis de 0.28190502506610815 con un p-value de 0.5954553058031636 entre los tipos de documentos ['Solicitud de copias certificadas'] y ['Manifestación de parte'].
Puesto que el p-value >= 0.05, NO podemos rechazar la hipótesis nula para la varibale X_apellido_actor. La variable X_num_juzgado NO conduce a diferencias estadísticamente significativas entre los tipos de documentos ['Solicitud de copias

,X_apellido_actor,X_apellido_autorizado,X_apellido_demandado,X_apellido_otro,X_nombre_actor,X_nombre_autorizado,X_nombre_demandado,X_nombre_otro,X_num_expediente,X_num_juzgado
['Solicitud de copias certificadas'] y ['Contestación de demanda'],"[2.015810276679856, 0.15566814689657382, 0.155...","[8.420396825396839, 0.003710348045150661, 0.00...","[5.100322932991402, 0.023921389538922824, 0.02...","[1.9115084050149131, 0.16679579882865997, 0.16...","[1.9209620607136206, 0.16575064215175345, 0.16...","[7.425047127284655, 0.00643221405346717, 0.006...","[5.100322932991402, 0.023921389538922824, 0.02...","[4.300893911283517, 0.03809234597123435, 0.038...","[13.630884893640001, 0.00022249498895590773, 0...","[4.410160889752745, 0.03572547152529185, 0.035..."
['Solicitud de copias certificadas'] y ['Manifestación de parte'],"[0.28190502506610815, 0.5954553058031636, 0.59...","[0.2634860395770986, 0.6077352506284788, 0.607...","[0.17366791074405652, 0.6768723820401589, 0.67...","[1.318958352571796, 0.2507790794048132, 0.2507...","[0.28190502506610815, 0.5954553058031636, 0.59...","[0.005697404515717999, 0.939831892177751, 0.93...","[0.17366791074405652, 0.6768723820401589, 0.67...","[2.8702496315812978, 0.09023087333742734, 0.09...","[7.978003652822997, 0.00473491278963161, 0.004...","[2.6740237691001605, 0.10199785457623392, 0.10..."
['Solicitud de copias certificadas'] y ['Consignación billete de deposito'],"[0.014876360911521913, 0.9029238162518338, 0.9...","[1.9285714285714028, 0.16491482255330014, 0.16...","[1.1377926421404623, 0.28611923693769326, 0.28...","[0.2649136577708031, 0.6067643247768499, 0.606...","[0.0852930852930901, 0.7702485484692947, 0.770...","[1.9285714285714028, 0.16491482255330014, 0.16...","[1.0097232610321558, 0.3149691462259916, 0.314...","[0.47021022455804795, 0.492890471322202, 0.492...","[0.11739130434782387, 0.7318812357199511, 0.73...","[0.6186895810956002, 0.4315346629850645, 0.431..."
['Contestación de demanda'] y ['Manifestación de parte'],"[6.045336856715785, 0.01394307255448336, 0.013...","[10.667670682730943, 0.0010902432331034134, 0....","[6.030413157326432, 0.01406144156693048, 0.014...","[0.34116541353383556, 0.5591572943909582, 0.55...","[5.55866965620329, 0.018389383355951916, 0.018...","[6.633636587164998, 0.010007075811427886, 0.01...","[5.073578042328039, 0.024293189857307208, 0.02...","[0.1843317972350453, 0.667676691138978, 0.6676...","[2.163019543077353, 0.14136665052764452, 0.141...","[0.9040835707502343, 0.3416891104887166, 0.341..."
['Contestación de demanda'] y ['Consignación billete de deposito'],"[8.761712679615137, 0.0030762003625622657, 0.0...","[13.052348993288616, 0.00030290427019869393, 0...","[0.17587217829898613, 0.6749448905244506, 0.67...","[0.2645446507515464, 0.6070149695389628, 0.607...","[13.716695437814028, 0.0002125565270770495, 0....","[11.726530612244913, 0.0006161535125042411, 0....","[0.04727785395624965, 0.8278695820050164, 0.82...","[0.42060278902384407, 0.516636417436005, 0.516...","[2.8099952221691415, 0.09367865471513297, 0.09...","[4.503958663221937, 0.03381648086311131, 0.033..."
['Manifestación de parte'] y ['Consignación billete de deposito'],"[0.2125845147219125, 0.6447491858863982, 0.644...","[0.9285714285713504, 0.3352344137959772, 0.335...","[0.9112752430815272, 0.3397762854898513, 0.339...","[1.0170316971164228, 0.3132241351189467, 0.313...","[0.4615384615384528, 0.49690584756477185, 0.49...","[1.9285714285714028, 0.16491482255330014, 0.16...","[0.8486706157872818, 0.3569286863065698, 0.356...","[1.0170316971164228, 0.3132241351189467, 0.313...","[2.7828213252514047, 0.09528018947949202, 0.09...","[4.732210755024448, 0.029602811761969418, 0.02..."


In [81]:
df_result = df_result.reset_index()
for i in df_result.columns:
    if i.startswith('X'):
        for ind in df_result.index:
            print(df_result[i][ind][1])
            df_result[i][ind] = df_result[i][ind][1]
df_result.to_excel('2_varianza.xlsx')

0.15566814689657382
0.5954553058031636
0.9029238162518338
0.01394307255448336
0.0030762003625622657
0.6447491858863982
0.003710348045150661
0.6077352506284788
0.16491482255330014
0.0010902432331034134
0.00030290427019869393
0.3352344137959772
0.023921389538922824
0.6768723820401589
0.28611923693769326
0.01406144156693048
0.6749448905244506
0.3397762854898513
0.16679579882865997
0.2507790794048132
0.6067643247768499
0.5591572943909582
0.6070149695389628
0.3132241351189467
0.16575064215175345
0.5954553058031636
0.7702485484692947
0.018389383355951916
0.0002125565270770495
0.49690584756477185
0.00643221405346717
0.939831892177751
0.16491482255330014
0.010007075811427886
0.0006161535125042411
0.16491482255330014
0.023921389538922824
0.6768723820401589
0.3149691462259916
0.024293189857307208
0.8278695820050164
0.3569286863065698
0.03809234597123435
0.09023087333742734
0.492890471322202
0.667676691138978
0.516636417436005
0.3132241351189467
0.00022249498895590773
0.00473491278963161
0.731881